# 实验2 基础爬虫实验

##  爬取酷狗TOP500的数据

### 1. 实验目的

1. 熟悉requests和BeautifulSoup库的基本用法
2. 掌握爬虫程序的开发流程
3. 掌握如何使用requests抓取网页内容
4. 掌握如何使用BeautifulSoup库解析HTML，采集目标数据
5. 掌握如何利用分页规律生成分页网址并抓取多页内容
6. 掌握如何爬取进入详细页面的链接网址，进而爬取详细页面中的目标数据
7. 掌握如何将爬取到的数据保存为CSV文件

### 2. 实验内容：

本实验爬取的内容为[酷狗榜单中酷狗TOP500](http://www.kugou.com/yy/rank/home/1-8888.html)的音乐信息

(1) 此站的分页规律如下（每页22首，共23页）：

1. http://www.kugou.com/yy/rank/home/1-8888.html
2. http://www.kugou.com/yy/rank/home/2-8888.html
3. http://www.kugou.com/yy/rank/home/3-8888.html
4. http://www.kugou.com/yy/rank/home/4-8888.html
5. http://www.kugou.com/yy/rank/home/5-8888.html

(2) 利用此分页规律爬取各分页中的音乐信息。

(3) 需要爬取的信息有：排名、歌手、歌曲名和歌曲时长。如下图所示：

![需获取的网页信息](images/kugou.jpeg)

(4) 将爬取到的音乐信息存储到名为kugouTop100.csv文件中

### 3. 实验代码

In [1]:
from myspider import HtmlParser
from bs4 import BeautifulSoup

class SougouTop500HtmlParser(HtmlParser):
    '''
    基于BeautifulSoup库的HTML解析器，用来解析搜狗TOP500网页数据
    '''
    def parse(self, html_content):
        
        # 内嵌函数，用于提取数据
        def parse_data(html_content):
            music_div = soup.find('div', class_='pc_temp_songlist')
            music_list = music_div.find_all('li')
            musics = []
            for music_li in music_list:
                rank = music_li.find('span', class_='pc_temp_num')
                rank = rank.text.strip()
                title = music_li.get('title')
                artist = title.split('-')[0].strip()
                title = title.split('-')[1].strip()
                time = music_li.find('span', class_='pc_temp_time')
                time = time.text.strip()
                music_info = {'rank': rank, 
                              'artist': artist, 
                              'title': title, 
                              'time': time
                             }
                musics.append(music_info)
            return musics
        # 内嵌函数，用于提取URL（本站无需提取URL）
        def parse_urls(html_content):
            pass
        
        if html_content is None:
            return None
        soup = BeautifulSoup(html_content, 'html.parser')
        # 提取数据
        data = parse_data(html_content)
        # 提取URL
        urls = parse_urls(html_content)
        
        return urls, data

In [7]:
from myspider import SpiderScheduler
from myspider import DataWriter

html_parser = SougouTop500HtmlParser()
data_writer = DataWriter('kugouTop100.csv')
spider = SpiderScheduler(html_parser,data_writer)
start_urls = ['http://www.kugou.com/yy/rank/home/{0}-8888.html'.format(i) for i in range(1,6)]
start_urls

['http://www.kugou.com/yy/rank/home/1-8888.html',
 'http://www.kugou.com/yy/rank/home/2-8888.html',
 'http://www.kugou.com/yy/rank/home/3-8888.html',
 'http://www.kugou.com/yy/rank/home/4-8888.html',
 'http://www.kugou.com/yy/rank/home/5-8888.html']

In [8]:
spider.crawl(start_urls)

正在爬取：http://www.kugou.com/yy/rank/home/3-8888.html...已完成1，剩余4.
正在爬取：http://www.kugou.com/yy/rank/home/5-8888.html...已完成2，剩余3.
正在爬取：http://www.kugou.com/yy/rank/home/1-8888.html...已完成3，剩余2.
正在爬取：http://www.kugou.com/yy/rank/home/4-8888.html...已完成4，剩余1.
正在爬取：http://www.kugou.com/yy/rank/home/2-8888.html...已完成5，剩余0.


In [9]:
!cat kugouTop100.csv

In [5]:
import myspider
help(myspider)

Help on module myspider:

NAME
    myspider - myspider：基础爬虫框架模块

DESCRIPTION
    基础爬虫框架主要包括五大模块，分别为：
    * 爬虫调度器（SpiderScheduler）：主要负责协调其他四个模块的工作
    * URL管理器（UrlManager）：负责管理URL链接
    * HTML下载器（HtmlDownloader）：负责下载HTML网页
    * HTML解析器（HtmlParser）：从下载的HTML网页中解析出新的URL链接和目标数据
    * 数据存储器（DataWriter）：用于将解析出来的目标数据保存到文件或数据库中

CLASSES
    builtins.object
        DataWriter
        HtmlDownloader
        HtmlParser
        SpiderScheduler
        UrlManager
    
    class DataWriter(builtins.object)
     |  DataWriter(filename)
     |  
     |  数据存储器：存储数据到文件或者数据库中
     |  默认实现，存储为CSV格式
     |  
     |  Methods defined here:
     |  
     |  __init__(self, filename)
     |      Initialize self.  See help(type(self)) for accurate signature.
     |  
     |  get(self)
     |      获取数据
     |      :return: list
     |  
     |  put(self, data)
     |      存放数据
     |      :param data:要存放的数据(list)
     |  
     |  save(self)
     |      存储数据
     |  
     |  ---------------------------------------

In [10]:
help(spider)

Help on SpiderScheduler in module myspider object:

class SpiderScheduler(builtins.object)
 |  SpiderScheduler(html_parser=None, data_writer=None)
 |  
 |  爬虫调度器：协调上述四个模块，爬取指定的URL列表
 |  
 |  Methods defined here:
 |  
 |  __init__(self, html_parser=None, data_writer=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  crawl(self, start_urls)
 |      从指定的初始URL列表中爬取数据
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



### 4. 代码分析

请在此处编写实验分析（双击编辑）

### 5. 实验总结

请在此处编写实验总结（双击编辑）